In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import csv

In [19]:
headers = {
    'authority': 'arxiv.org',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'cache-control': 'no-cache',
    'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
    
title_list = []
author_list = []
subject_list = []
abstract_list = []

In [20]:
def scrape_page(soup,headers):
    
    articles = soup.find_all('div', class_='meta')
    for article in articles:
        
        title = article.find('div', class_='list-title mathjax').text.strip()
        title = title.replace('Title:', '').strip()
        title_list.append(title)
        
        author = article.find('div', class_='list-authors').text.strip()
        author = author.replace('Authors:', '').strip()
        author_list.append(author)
        
        subject = article.find('div', class_='list-subjects').text.strip()
        subject = subject.replace('Subjects:', '').strip()
        subject_list.append(subject)

    links = soup.find_all('a', title='Abstract')

    for link in tqdm(links, desc="descending"): 
        paper_url = "https://arxiv.org" + link['href']  
        paper_page = requests.get(paper_url, headers=headers)
        paper_soup = BeautifulSoup(paper_page.content, 'html.parser')
        abstract = paper_soup.find('blockquote', class_='abstract').text.strip()  
        abstract = abstract.replace('Abstract:', '').strip()
        abstract_list.append(abstract)
        
    return title_list, author_list, subject_list, abstract_list

In [13]:
url = "https://arxiv.org/list/cs/pastweek?skip=0&show=25"
total_papers = 100  
papers_per_page = 25  
num_pages = int(total_papers / papers_per_page)

for page_num in range(num_pages):
    page_url = f"{url}&skip={page_num * papers_per_page}"
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    scrape_page(soup, headers)

    time.sleep(1)
    

提取进度: 100%|█████████████████████████████████| 25/25 [00:48<00:00,  1.96s/it]


In [18]:
data = list(zip(title_list, author_list, subject_list, abstract_list))
data = data[:100]
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Title', 'Author', 'Subject', 'Abstract'])  # 写入表头
    writer.writerows(data) 